In [2]:
import cv2
import pandas as pd
import numpy as np

from pathlib import Path

In [3]:
a4c_dir = Path("/home/lex/Development/echonet-dynamic/a4c-video-dir")
video_dir = a4c_dir / "Videos"
meta_fp = a4c_dir / "FileList.csv"

In [4]:
meta_df = pd.read_csv(meta_fp)
video_files = [v for v in video_dir.iterdir()]
video_file = video_files[0]

In [5]:
print(meta_df.head())

             FileName         EF         ESV         EDV  FrameHeight  \
0  0X100009310A3BD7FC  78.498406   14.881368   69.210534          112   
1  0X1002E8FBACD08477  59.101988   40.383876   98.742884          112   
2  0X1005D03EED19C65B  62.363798   14.267784   37.909734          112   
3  0X10075961BC11C88E  54.545097   33.143084   72.914210          112   
4  0X10094BA0A028EAC3  24.887742  127.581945  169.855024          112   

   FrameWidth  FPS  NumberOfFrames  Split  
0         112   50             174    VAL  
1         112   50             215  TRAIN  
2         112   50             104  TRAIN  
3         112   55             122  TRAIN  
4         112   52             207    VAL  


In [49]:
def watershed(frame):
    """Stolen from: https://docs.opencv2.org/4.x/d3/db4/tutorial_py_watershed.html"""
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=1)
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    markers = cv2.watershed(frame,markers)
    new_frame = frame.copy()
    new_frame[markers == -1] = [255,0,0]
    return new_frame, thresh, sure_bg, unknown


In [52]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(str(video_file))
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
 
# Read until video is completed
try:
    cv2.namedWindow("Frame + Watershed", cv2.WINDOW_NORMAL)
    cv2.namedWindow("Intermediate", cv2.WINDOW_NORMAL)
    frame_size = 300
    cv2.resizeWindow("Frame + Watershed", 2*frame_size, frame_size)
    cv2.resizeWindow("Intermediate", 3*frame_size, frame_size)

    start_x = 350
    start_y = 180
    cv2.moveWindow("Frame + Watershed", start_x + frame_size//2, start_y)
    cv2.moveWindow("Intermediate", start_x, int(start_y+1.1*frame_size))

    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:
            watershed_frame, thresh, sure_bg, unknown = watershed(frame)

            # height, width, channels = frame.shape
            # print(frame.shape)

            frame_and_watershed = np.concatenate([frame, watershed_frame], axis=1)
            intermediate = np.concatenate([thresh, sure_bg, unknown], axis=1)

            # Display the resulting frame
            cv2.imshow("Frame + Watershed",frame_and_watershed)
            cv2.imshow("Intermediate", intermediate)
            print(cv2.getWindowImageRect("Frame + Watershed"))
            print(cv2.getWindowImageRect("Intermediate"))

            # Press Q on keyboard to  exit
            keypress = cv2.waitKey(25) & 0xFF
            if keypress == ord('q'):
                break
            elif keypress == ord(' '):
                print("Press space again to resume")
                while True:
                    second_keypress = cv2.waitKey(0) & 0xFF
                    if second_keypress == ord(' '):
                        break
        
        # Break the loop
        else: 
            break
finally: 
    # When everything done, release the video capture object
    cap.release()
    
    # Closes all the frames
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread (0x1d7de20).
Cannot move to target thread (0x165c830)

QObject::moveToThread: Current thread (0x165c830) is not the object's thread

(500, 286, 600, 171)
(359, 538, 882, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 756, 252)
(548, 245, 504, 252)
(422, 538, 75